[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/camenduru/stable-video-diffusion-colab/blob/main/stable_video_diffusion_fp32_colab.ipynb)

In [ ]:
%cd /content
!git clone -b dev https://github.com/camenduru/stable-video-diffusion-hf
%cd /content/stable-video-diffusion-hf
!pip install -r https://github.com/camenduru/stable-video-diffusion-colab/raw/main/requirements.txt
!python app.py